# `CLOUD.jl` - 2D linear advection example

In [20]:
using CLOUD, OrdinaryDiffEq

Define the physical problem

In [21]:
θ = π/4 # wave angle
a = sqrt(2) # wave speed
A = 1.0  # amplitude
L = 1.0  # domain length
k = (2*π/L, 2*π/L)  # wave number
T = 1.0/(a*max(abs(cos(θ)),abs(cos(θ)))) # end time for one period

initial_data = InitialDataSine(A,k)
conservation_law = LinearAdvectionEquation((a*cos(θ),a*sin(θ)));

Set discretization parameters

In [22]:
M = 4
p = 4
p_map = 4
form = WeakConservationForm(StandardMapping(),LaxFriedrichsNumericalFlux())
strategy = Lazy()

ode_algorithm = CarpenterKennedy2N54()
dt = 0.1*(L/M)/(a*(2*p+1));
write_interval = floor(Int, T/(dt*10));

## Collocated DGSEM on quadrilaterals

Set up a `DGSEM` scheme with LG quadrature on quadrilaterals of degree `p` with `M` edges in each direction

In [23]:
reference_approximation =ReferenceApproximation(
    DGSEM(p), Quad(),
    volume_quadrature_rule=LGQuadrature(),
    facet_quadrature_rule=LGQuadrature(),
    mapping_degree=p_map, N_plot=50)

mesh = warp_mesh(uniform_periodic_mesh(
    reference_approximation.reference_element, 
    ((0.0,L),(0.0,L)), (M,M)), 
    reference_approximation.reference_element, 0.1)

spatial_discretization = SpatialDiscretization(mesh, 
    reference_approximation)

results_path = save_project(conservation_law,
     spatial_discretization, initial_data, form, (0.0, T), Lazy(),
     "results/advection_2d_dgsem/", overwrite=true, clear=true)

"results/advection_2d_dgsem/"

Run the solver

In [24]:
ode_problem = semidiscretize(conservation_law,
    spatial_discretization,
    initial_data, 
    form,
    (0.0, T),
    strategy)

save_solution(ode_problem.u0, 0.0, results_path, 0)

CLOUD_reset_timer()
sol = solve(ode_problem, ode_algorithm, adaptive=false, dt=dt,
    save_everystep=false, callback=save_callback(results_path, write_interval))
CLOUD_print_timer()

save_solution(last(sol.u), last(sol.t), results_path, "final")

 ────────────────────────────────────────────────────────────────────────────────
                                        Time                    Allocations      
                               ───────────────────────   ────────────────────────
       Tot / % measured:         78.1μs / 717362.7%        10.6KiB / 6255254.0%  

 Section               ncalls     time    %tot     avg     alloc    %tot      avg
 ────────────────────────────────────────────────────────────────────────────────
 eval residual          40.8k    371ms   66.2%  9.09μs    293MiB   45.4%  7.34KiB
   volume terms         40.8k    181ms   32.3%  4.44μs    156MiB   24.1%  3.91KiB
   facet terms          40.8k   95.0ms   16.9%  2.33μs   86.6MiB   13.4%  2.17KiB
   mass matrix solve    40.8k   42.9ms    7.6%  1.05μs   19.9MiB    3.1%     512B
 extrap solution        40.8k   88.2ms   15.7%  2.16μs   71.0MiB   11.0%  1.78KiB
 eval num flux          40.8k   40.5ms    7.2%   993ns    106MiB   16.4%  2.66KiB
 eval flux     

Calculate the integral L<sup>2</sup> error norm and conservation/energy error

In [25]:
error_analysis = ErrorAnalysis(results_path, conservation_law, 
    spatial_discretization)
conservation_analysis = PrimaryConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)
energy_analysis = EnergyConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)

println("L2 error:")
println(analyze(error_analysis, last(sol.u), initial_data)...)
println("Conservation (initial/final/diff):")
println(analyze(conservation_analysis)...)
println("Energy (initial/final/diff):")
println(analyze(energy_analysis)...)

L2 error:
0.00068216127441316
Conservation (initial/final/diff):
[-1.108594540819019e-8][-1.1085946192285201e-8][-7.840950111415168e-16]
Energy (initial/final/diff):
[0.12499998811286066][0.12497973419588697][-2.025391697368928e-5]


## Modal DG scheme on triangles

Set up a `DGMulti` scheme on triangles of degree `p` with `M` edges in each direction

In [26]:
reference_approximation = ReferenceApproximation(
    DGMulti(p), Tri(), mapping_degree=p_map, N_plot=10)

mesh = warp_mesh(uniform_periodic_mesh(
    reference_approximation.reference_element, 
    ((0.0,L),(0.0,L)), (M,M)), 
    reference_approximation.reference_element, 0.1)

spatial_discretization = SpatialDiscretization(mesh, 
    reference_approximation)

results_path = save_project(conservation_law,
     spatial_discretization, initial_data, form, (0.0, T), Lazy(),
     "results/advection_2d_dgmulti/", overwrite=true, clear=true)

"results/advection_2d_dgmulti/"

Run the solver

In [27]:
ode_problem = semidiscretize(conservation_law,
    spatial_discretization,
    initial_data, 
    form,
    (0.0, T),
    strategy)

save_solution(ode_problem.u0, 0.0, results_path, 0)

CLOUD_reset_timer()
sol = solve(ode_problem, ode_algorithm, adaptive=false, dt=dt,
    save_everystep=false, callback=save_callback(results_path, write_interval))
CLOUD_print_timer()

save_solution(last(sol.u), last(sol.t), results_path, "final")

 ────────────────────────────────────────────────────────────────────────────────
                                        Time                    Allocations      
                               ───────────────────────   ────────────────────────
       Tot / % measured:         78.7μs / 1186905.3%      10.6KiB / 10385157.0%  

 Section               ncalls     time    %tot     avg     alloc    %tot      avg
 ────────────────────────────────────────────────────────────────────────────────
 eval residual          81.6k    521ms   55.8%  6.39μs    453MiB   42.3%  5.69KiB
   volume terms         81.6k    193ms   20.7%  2.37μs    206MiB   19.2%  2.58KiB
   mass matrix solve    81.6k    187ms   20.0%  2.29μs    186MiB   17.3%  2.33KiB
   facet terms          81.6k   29.4ms    3.2%   361ns   13.7MiB    1.3%     176B
 eval flux              81.6k    132ms   14.2%  1.62μs   69.8MiB    6.5%     896B
 gather ext state       81.6k   96.7ms   10.4%  1.19μs    350MiB   32.7%  4.39KiB
 extrap solutio

Calculate the integral L<sup>2</sup> error norm and conservation/energy error

In [28]:
error_analysis = ErrorAnalysis(results_path, conservation_law, 
    spatial_discretization)
conservation_analysis = PrimaryConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)
energy_analysis = EnergyConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)

println("L2 error:")
println(analyze(error_analysis, last(sol.u), initial_data)...)
println("Conservation (initial/final/diff):")
println(analyze(conservation_analysis)...)
println("Energy (initial/final/diff):")
println(analyze(energy_analysis)...)

L2 error:
0.0034594895345242908
Conservation (initial/final/diff):
[-1.4530229013823975e-7][-1.4530229063480435e-7][-4.96564594998361e-16]
Energy (initial/final/diff):
[0.12499475017905026][0.12485531367220426][-0.0001394365068460024]
